<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-f26d2da9699a3d4e'
PROJECT = 'qwiklabs-gcp-f26d2da9699a3d4e'
REGION = 'us-east1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq

query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

In [5]:
print("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth")

SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
) GROUP BY hashmonth


There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [6]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,1088037545023002395,362125
1,1525201076796226340,303664
2,3408502330831153141,323970
3,4979697502521811334,357965
4,411066950820961322,318834


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [7]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13413 examples in the train dataset and 3404 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [8]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,3.937456,False,17,2,35.0,774501970389208065
1,7.874912,True,19,1,37.0,774501970389208065
2,6.874013,False,31,1,37.0,774501970389208065
3,6.638119,False,19,1,40.0,774501970389208065
4,9.563653,True,31,1,39.0,774501970389208065


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [9]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13399.000000,13413.000000,13413.000000,13312.000000,1.341300e+04
mean,7.231215,27.450682,1.035637,38.610276,4.402357e+18
std,1.316427,6.150250,0.195186,2.571056,2.802992e+18
min,0.526905,12.000000,1.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,1.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,1.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,1.000000,40.000000,7.108882e+18
max,12.500210,54.000000,4.000000,47.000000,9.183606e+18


In [10]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [11]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,3.937456,False,17,Twins(2),35.0,774501970389208065
1,7.874912,True,19,Single(1),37.0,774501970389208065
2,6.874013,False,31,Single(1),37.0,774501970389208065
3,6.638119,False,19,Single(1),40.0,774501970389208065
4,9.563653,True,31,Single(1),39.0,774501970389208065


In [12]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13408,8.216629,Unknown,18,Single(1),40.0,6637442812569910270
13409,4.828124,Unknown,15,Single(1),31.0,6637442812569910270
13410,8.562754,Unknown,24,Single(1),38.0,6637442812569910270
13411,7.500126,Unknown,31,Single(1),39.0,6637442812569910270
13412,8.375361,Unknown,33,Single(1),37.0,6637442812569910270


In [13]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26606.000000,26606.000000,26606.000000,2.660600e+04
mean,7.232541,27.453883,38.613020,4.399203e+18
std,1.315492,6.139918,2.565347,2.801013e+18
min,0.526905,12.000000,18.000000,1.244589e+17
25%,6.563162,23.000000,38.000000,1.525201e+18
50%,7.312733,27.000000,39.000000,4.329667e+18
75%,8.062305,32.000000,40.000000,7.108882e+18
max,12.500210,54.000000,47.000000,9.183606e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [14]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [15]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6742 eval.csv
  26606 train.csv
  33348 total
==> eval.csv <==
8.000575487979999,True,34,Single(1),38.0,2246942437170405963
6.3118345610599995,True,22,Single(1),42.0,6244544205302024223
6.393405598,False,28,Single(1),36.0,6782146986770280327
7.4626475687,False,24,Single(1),38.0,6244544205302024223
8.062304921339999,True,25,Single(1),39.0,3182182455926341111
8.99926953484,True,44,Single(1),40.0,7146494315947640619
6.5477291814,False,20,Single(1),38.0,1569531340167098963
6.7483498398199995,False,22,Single(1),37.0,6910174677251748583
5.3572329666,True,19,Single(1),41.0,1866590652208008467
6.59843550166,False,28,Single(1),41.0,6910174677251748583

==> train.csv <==
3.93745599932,False,17,Twins(2),35.0,774501970389208065
7.87491199864,True,19,Single(1),37.0,774501970389208065
6.87401332916,False,31,Single(1),37.0,774501970389208065
6.6381187088199995,False,19,Single(1),40.0,774501970389208065
9.56365292556,True,31,Single(1),39.0,774501970389208065
7.73381615096,False,29,Single(1),34.0,77

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License